# Lab01 - Damped Spring - Lab Bench

## Overview

Labs 1 & 2 will allow you to experiment (physically and numerically)
with two dynamical systems studied in Week 2: the mass-spring and
pendulum oscillators.

-   set up and take video of the spring system
-   extract position data from a short video of a damped spring–mass
    oscillator using [Tracker](https://physlets.org/tracker/), a
    miraculous (if slightly cumbersome) piece of software
-   use plots to infer what the system depends on (its **state**)
-   connect observations to a first-order dynamical system
-   practice fixed-point analysis

------------------------------------------------------------------------

## Part II: Investigating the system

### Timeseries investigation

To investigate the motion of the spring, you are going to look at both
the data you collected and also use that data to estimate parameters so
that you can simulated a longer time series from equations.

First we need to load some packages:

In [ ]:
# Imports used throughout the lab (arrays, plotting, data frames, ODE integration)
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.integrate import solve_ivp
import datetime

notebook_loc = Path(os.getcwd())
# check that has the right parent
if notebook_loc.parent.name != 'work':
  print('Please make a copy of this notebook and put it in your "work" folder!')

fig_dir = Path('./figures')
fig_dir.mkdir(parents=True, exist_ok=True)

def fig_save(fig_name, filetype='png'):
  fig_path = fig_dir/f'{fig_name}.{filetype}'
  unique_fig_path = fig_dir/f'{fig_path.name}__{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}{fig_path.suffix}'
  plt.savefig(fig_path, dpi=300)
  plt.savefig(unique_fig_path, dpi=300)


#### Load data

Load your data using the code below.

In [ ]:
# Load in csv
data_dir = Path('data')
csv_file_name = ".csv" # you need to edit this file name!
df_obs = pd.read_csv(data_dir/csv_file_name) # , skiprows=1
df_obs['treatment'] = 'observed'
df_obs = df_obs.rename(columns={'t':'time', 'v_{y}':'v'})

df_obs.head() #if this doesn't look right, you may need to tweak pd.read_csv to skip a row

Then specify the values of your system:

In [ ]:
# physical parameters
m =    # mass
k =     # spring constant

t_start = 0
t_end = df_obs['time'].max()- df_obs['time'].min()# seconds of data collection

You will use `run_shm` to simulate the simple harmonic motion of your
spring. It takes in the spring parameters, set up the ODE, solve it, and
return a `DataFrame` with containing the position and velocity
timeseries.

In [ ]:
def run_shm(t, t_start, t_end, y0, c, m,k, label='simulated'):

  def damped_spring(t, y):
      y, v = y
      dydt = v
      dvdt = -(c/m)*v - (k/m)*y
      return [dydt, dvdt]

  # Solve the ODE
  sol = solve_ivp(damped_spring, (t_start, t_end), y0, t_eval=t)

  # Extract position (x) and velocity (v) from the solution
  y_sim = sol.y[0]
  v_sim = sol.y[1]

  if label == 'simulated':
    label = f'simulated, c={c}, k={k}'

  df_sim = make_df(t, y_sim, v_sim, label=label)
  return df_sim, y_sim, v_sim

def make_df(t, y, v, label=None):

  # package into a dataframe
  df_sim = pd.DataFrame({'time': t, 'y':y, 'v':v})
  df_sim['treatment'] = label

  return df_sim

This sets inital conditions and time span

In [ ]:
init_displacement = df_obs['y'].min()   # initial position (m)
print(f'initial displacement: {df_obs["y"].min()}m')
v0 = 0.0   # initial velocity (m/s)
y0 = [init_displacement, v0] # initial state vector

t = np.linspace(t_start, t_end, len(df_obs)) # len(df_obs) is the number of points from your observation timeseries. You may choose to adjust this in future.

#### Simulate the system

Now play with the damping until it emulates your physical setup. Check
your choices by plotting both together for the same time window.

In [ ]:
# Choose a damping coefficient (tune this to match your observed motion)
c =     # damping coefficient, start with .5

df_sim, y_sim, v_sim = run_shm(t, t_start, t_end, y0, c, m,k )

#### 1. Plot position vs time

Plot position vs. time for observed vs simulated trajectories

In [ ]:
df = pd.concat([df_obs, df_sim])

fig, ax = plt.subplots(figsize=(10, 5))
sns.lineplot(data = df, x='time', y = 'y', hue='treatment', ax=ax, alpha=.5)
ax.set_ylabel('Position (m)')
ax.set_title('Simple Harmonic Motion of a Spring-Mass System')
ax.legend()

plt.tight_layout()
# fig_save('spring_obs_sim', filetype='png')

If you are happy with your choice of damping coefficient, fiddle with
the value of `t_end` in the code below until the amplitude of your
simulated displacement nears 0.

In [ ]:
t_end = 
t = np.linspace(t_start, t_end, 3*len(df_obs))
df_sim, y_sim, v_sim = run_shm(t, t_start, t_end, y0, c, m,k )

df = pd.concat([df_obs, df_sim])

fig, ax = plt.subplots(figsize=(10, 5))
sns.lineplot(data = df, x='time', y = 'y', hue='treatment', ax=ax, alpha=.5)
ax.set_ylabel('Position (m)')
ax.set_title('Simple Harmonic Motion of a Spring-Mass System')
ax.legend()

plt.tight_layout()

#### 2. Does behavior depend on time? Period across cycles

Estimate the period for each cycle (e.g., peak-to-peak time
differences).

The code below identifies peak times (local maxima) and estimates the
oscillation period from peak-to-peak spacing

In [ ]:
from scipy.signal import argrelextrema

df_maxes_lst = []

for treatment, grp_df in df.groupby('treatment'):

  ilocs_max = argrelextrema(grp_df.y.values, np.greater_equal, order=3)[0]
  df_maxes_tmp = pd.DataFrame(
      {'time':grp_df.iloc[ilocs_max]['time'],
       'y':grp_df.iloc[ilocs_max]['y']})
  df_maxes_tmp['treatment'] = treatment
  df_maxes_lst.append(df_maxes_tmp)

df_maxes = pd.concat(df_maxes_lst)

df_periods = df_maxes.copy()
df_periods['period'] = df_periods['time']
df_periods = df_periods.set_index(['time', 'treatment'], drop =True).groupby(level= 1).diff().dropna()
df_periods = df_periods.reset_index()
print(df_periods.head())

Visualize the distribution of estimated periods using the code below.
After you have run it, try specifying `hue='treatment'` and run it
again.

Hint: to specify a parameter, put a comma after the last named argument
(in this case `x='period'`) and add your new parameter as `param=value`
(in this case, `hue='treatment'`)

In [ ]:
sns.histplot(data = df_periods, x='period')

Use the formula from your notes to calculate period.

In [ ]:
period = 

The code below plots the timeseries with the detected peaks (good sanity
check), and the histograms with the calculated value.

In [ ]:
fig, axs = plt.subplot_mosaic([['scatter_line', 'hist']],
                              figsize=(12, 5),
                              width_ratios=(4, 1),
                              layout='constrained')

df = pd.concat([df_obs, df_sim])
axs['scatter_line'] = sns.lineplot(data = df, x='time', y='y', hue= 'treatment', ax=axs['scatter_line'])

axs['scatter_line'] = sns.scatterplot(data = df_maxes, x='time', y='y', hue='treatment', ax=axs['scatter_line'],**{'zorder':10})
axs['scatter_line'].legend()
axs['scatter_line'].set_xlabel('Time (s)')
axs['scatter_line'].set_ylabel('Position (cm)')

sns.histplot( data = df_periods, x = 'period', hue='treatment', ax=axs['hist'])
axs['hist'].axvline(x = period, c='k', label='calculated')
axs['hist'].annotate('calculated', xy=(period, 17), xytext=(1, 19),
            arrowprops=dict(facecolor='black', shrink=0.05),
            )
axs['hist'].set_xlabel('Period (s)')
plt.tight_layout()

# fig_save('ts_with_peaks__hist')

------------------------------------------------------------------------

#### 5. Phase portrait from simulation

Dynamical systems are particularly interesting because they have
structure that doesn’t depend on time. Indeed, they are surprisingly
predictable if you frame them correctly. As a general rule, the next
state of a dynamical system is a function of its currrent state. There
are some ways of getting at that in the time domain which you’ll hear
about later in the semester, but if we have differnetial equations, we
can get an overview of how the system behaves without analyzing
timeseries (which are not always orderly to the naked eye).

Dynamical systems are interesting because, at their core, they evolve
according to simple rules: what happens next depends on where the system
is now. That idea shows up in many contexts, from mechanical systems to
climate models to population dynamics.

When we look at these systems in time, that structure is not always
obvious. Time series can be noisy, oscillatory, or hard to interpret by
eye. If we instead keep track of the variables that describe the system
at each moment, we can often see the behavior more clearly.

A phase portrait shows how the system moves through a space defined by
its variables—in this case, position and velocity. As the system
evolves, it traces out a path that reflects the underlying dynamics,
independent of time. Patterns like damping, stability, and long-term
behavior often become clear in this view.

In this section, you will construct a phase portrait of the damped
spring. First, run the code as written to plot the trajectory in (y,v)
space.

In [ ]:
# --- Phase portrait: trajectory + optional vector field + streamlines ---

t_end = 240
t = np.linspace(t_start, t_end, len(df_obs) * 6)  # adjust density later if you want
c = 0.15

df_sim2, y_sim, v_sim = run_shm(t, t_start, t_end, y0, c, m, k)

# Optionally trim to later times (keep as-is for now)
df_sim_sub = df_sim2  # or: df_sim2[df_sim2["time"] > 200].copy()

# --- pick data source for trajectory ---
y_traj = df_sim_sub["y"].to_numpy()
v_traj = df_sim_sub["v"].to_numpy()

fig, ax = plt.subplots()
# --- 1) trajectory only ---
ax.plot(y_traj, v_traj)
sns.scatterplot(data=df_sim_sub, x='y', y='v', ax=ax, s=0.75)

ax.set_title("Phase portrait: trajectory")
ax.spines[['top', 'right']].set_visible(False)
ax.spines[['left', 'bottom']].set_position('zero')
ax.set_xticks(ax.get_xlim())
ax.set_yticks(ax.get_ylim())

ax.xaxis.set_label_coords(1.05, 0.5) # Adjust coords as needed
ax.yaxis.set_label_coords(0.5, -.02) # Adjust coords as needed

# # --- 2) add a vector field (uncomment to show it), then replot ---
# def spring_rhs(y, v, c, m, k):
#     dy = v
#     dv = -(c/m) * v - (k/m) * y
#     return dy, dv

# # --- vector field grid (controls: bounds and density) ---
# y_min, y_max = ax.get_xlim()#-2, 2
# v_min, v_max = ax.get_ylim()#-3, 3
# ny, nv = 21, 21  # grid density

# Y, V = np.meshgrid(np.linspace(y_min, y_max, ny),
#                    np.linspace(v_min, v_max, nv))

# dY, dV = spring_rhs(Y, V, c, m, k)

# # Normalize arrows so direction is visible everywhere (optional but usually better)
# speed = np.hypot(dY, dV)
# dYn = dY / (speed + 1e-12)
# dVn = dV / (speed + 1e-12)

# # Uncomment to show arrows:
# ax.quiver(Y, V, dYn, dVn, angles="xy", alpha=.5)
# ax.set_title("Phase portrait: trajectory + vector field")
plt.tight_layout

# fig_save('phase_portrait')

Now uncomment the bottom section of the phase portrait code, run it
again, and run the cell below it.

In [ ]:
import phaseportrait

def damped_spring(y, v):
    # captures c, m, k from the outer scope
    return v, -(c/m)*v - (k/m)*y

pp = phaseportrait.PhasePortrait2D(
    damped_spring,
    [-2, 2],          # x-range (position y)
    ylim=[-3, 3],     # y-range (velocity v)
    Title="Streamlines",
    xlabel="y",
    ylabel="v",
)

fig, ax = pp.plot()   # if this errors, try: pp.plot()
ax.spines[['top', 'right', 'left', 'bottom']].set_visible(False)

# fig_save('streamlines')